In [ ]:
# Collect Data
'''
Records screen from the game and saves it to a file
- GTA V 
- Windowed mode
- Resolution: 800 x 600
- Keep window at top left corner of screen (0,0)
'''

In [ ]:
# python 3.7.9
import numpy as np
print(np.__version__) #1.18.5

In [ ]:
import os
import time
import cv2
import numpy as np
from grabscreen import grab_screen
from getkeys import key_check

key_map = {
    'W': [1, 0, 0, 0, 0, 0, 0, 0, 0],
    'S': [0, 1, 0, 0, 0, 0, 0, 0, 0],
    'A': [0, 0, 1, 0, 0, 0, 0, 0, 0],
    'D': [0, 0, 0, 1, 0, 0, 0, 0, 0],
    'WA': [0, 0, 0, 0, 1, 0, 0, 0, 0],
    'WD': [0, 0, 0, 0, 0, 1, 0, 0, 0],
    'SA': [0, 0, 0, 0, 0, 0, 1, 0, 0],
    'SD': [0, 0, 0, 0, 0, 0, 0, 1, 0],
    'NK': [0, 0, 0, 0, 0, 0, 0, 0, 1],
} # key map for the output

starting_value = 176
BATCH_SIZE = 5000 # ~370Mb/1000 images
mode = 'RGB' #'RGB' or 'GRAY'

while True:
    file_name = 'training_data-{0}.npy'.format(starting_value)
    if os.path.isfile(file_name):
        print('File exists, moving along', starting_value)
        starting_value += 1
    else:
        print('File does not exist, starting fresh!', starting_value)
        break

def keys_to_output(keys):
    '''
    Convert keys to a ...multi-hot... array
     0  1  2  3  4   5   6   7    8 
    [W, S, A, D, WA, WD, SA, SD, NOKEY ] boolean values.
    '''
    if len(keys) == 0:
        return key_map['NK']
    
    elif len(keys) == 1:
        if keys[0] == 'W':
            return key_map['W']
        elif keys[0] == 'S':
            return key_map['S']
        elif keys[0] == 'A':
            return key_map['A']
        elif keys[0] == 'D':
            return key_map['D']
        
    elif len(keys) == 3:
        return key_map[keys[-1]]

def main(file_name, starting_value, mode, BATCH_SIZE):
    training_data = []
    for i in range(10, 0, -1):
        print(i + 1)
        time.sleep(1)
    paused = False
    print('STARTING!!!')

    while True:
        if not paused:
            screen = grab_screen(region=(0, 40, 800, 640))
            # region defines the area of the screen to capture

            screen = cv2.resize(screen, (480, 270))
            # resize the screen to 480x270

            #mode = cv2.COLOR_BGR2RGB if mode == 'RGB' else cv2.COLOR_BGR2GRAY
            mode = cv2.COLOR_BGR2RGB
            screen = cv2.cvtColor(screen, mode)
            # convert the screen from BGR to color mode

            keys = key_check() # list of keys pressed
            
            output = keys_to_output(keys)
            training_data.append([screen, output])
            # append the screen and the output to the training data

            if len(training_data) % 1000 == 0:
                print(len(training_data))
                if len(training_data) == BATCH_SIZE:
                    np.save(file_name, np.array(training_data))
                    print('SAVED')
                    training_data = []
                    starting_value += 1
                    file_name = 'training_data-{0}.npy'.format(starting_value)

        keys = key_check()

        if 'T' in keys:
            if paused:
                paused = False
                print('Unpaused!')
                time.sleep(1)
            else:
                print('Pausing!')
                paused = True
                time.sleep(1)

if __name__ == "__main__":
    main(file_name, starting_value, mode, BATCH_SIZE)

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from random import shuffle

train_data = np.load(r'C:\Local Disk D\gtav\self-driving_GTAV\mini\training_data-mini.npy', allow_pickle=True)  

df = pd.DataFrame(train_data)
print(df.head())
print(Counter(df[1].apply(str)))

w = []
s =[]
a = []
d = []
wa = []
wd = []
sa = []
sd = []
nk = []

# 'W': [1, 0, 0, 0, 0, 0, 0, 0, 0],
# 'S': [0, 1, 0, 0, 0, 0, 0, 0, 0],
# 'A': [0, 0, 1, 0, 0, 0, 0, 0, 0],
# 'D': [0, 0, 0, 1, 0, 0, 0, 0, 0],
# 'WA': [0, 0, 0, 0, 1, 0, 0, 0, 0],
# 'WD': [0, 0, 0, 0, 0, 1, 0, 0, 0],
# 'SA': [0, 0, 0, 0, 0, 0, 1, 0, 0],
# 'SD': [0, 0, 0, 0, 0, 0, 0, 1, 0],
# 'NK': [0, 0, 0, 0, 0, 0, 0, 0, 1],
# 'default': [0, 0, 0, 0, 0, 0, 0, 0, 0],

for data in train_data:
    img = data[0]
    choice = data[1]

    if choice == [1, 0, 0, 0, 0, 0, 0, 0, 0]:
        w.append([img, choice])
    elif choice == [0, 1, 0, 0, 0, 0, 0, 0, 0]:
        s.append([img, choice])
    elif choice == [0, 0, 1, 0, 0, 0, 0, 0, 0]:
        a.append([img, choice])
    elif choice == [0, 0, 0, 1, 0, 0, 0, 0, 0]:
        d.append([img, choice])
    elif choice == [0, 0, 0, 0, 1, 0, 0, 0, 0]:
        wa.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 1, 0, 0, 0]:
        wd.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 0, 1, 0, 0]:
        sa.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 0, 0, 1, 0]:
        sd.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 0, 0, 0, 1]:
        nk.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 0, 0, 0, 0]:
        pass    
    else:
        print('no matches')    

print(len(w), len(s), len(a), len(d), len(wa), len(wd), len(sa), len(sd), len(nk))

final_data = w + a + d
shuffle(final_data)
print(len(final_data))

np.save('training_data_raw.npy', final_data)
print('done')


                                                   0  \
0  [[[75, 74, 72], [225, 220, 216], [225, 220, 21...   
1  [[[75, 73, 72], [225, 220, 216], [225, 220, 21...   
2  [[[75, 73, 72], [225, 220, 216], [225, 220, 21...   
3  [[[75, 73, 72], [224, 220, 215], [224, 220, 21...   
4  [[[75, 73, 72], [224, 220, 215], [224, 220, 21...   

                             1  
0  [1, 0, 0, 0, 0, 0, 0, 0, 0]  
1  [1, 0, 0, 0, 0, 0, 0, 0, 0]  
2  [1, 0, 0, 0, 0, 0, 0, 0, 0]  
3  [1, 0, 0, 0, 0, 0, 0, 0, 0]  
4  [1, 0, 0, 0, 0, 0, 0, 0, 0]  
Counter({'[1, 0, 0, 0, 0, 0, 0, 0, 0]': 3627, '[0, 0, 0, 0, 0, 1, 0, 0, 0]': 416, '[0, 0, 0, 0, 1, 0, 0, 0, 0]': 364, '[0, 0, 0, 0, 0, 0, 0, 0, 1]': 248, '[0, 0, 0, 1, 0, 0, 0, 0, 0]': 106, '[0, 0, 1, 0, 0, 0, 0, 0, 0]': 104, '[0, 1, 0, 0, 0, 0, 0, 0, 0]': 50, '[0, 0, 0, 0, 0, 0, 0, 1, 0]': 47, '[0, 0, 0, 0, 0, 0, 1, 0, 0]': 35, 'None': 3})
no matches
no matches
no matches
3627 50 104 106 364 416 35 47 248
3837
done


print(np.__version__)

In [2]:
print(np.__version__)

1.18.5


In [ ]:
# balance_data.py

import numpy as np
import pandas as pd
from collections import Counter
from random import shuffle

train_data = np.load('training_data-1.npy', allow_pickle=True)  
i = 13

df = pd.DataFrame(train_data)
print(Counter(df[1].apply(str)))

w = []
s =[]
a = []
d = []
wa = []
wd = []
sa = []
sd = []
nk = []

for data in train_data:
    img = data[0]
    choice = data[1]

    if choice == [1, 0, 0, 0, 0, 0, 0, 0, 0]:
        w.append([img, choice])
    elif choice == [0, 1, 0, 0, 0, 0, 0, 0, 0]:
        s.append([img, choice])
    elif choice == [0, 0, 1, 0, 0, 0, 0, 0, 0]:
        a.append([img, choice])
    elif choice == [0, 0, 0, 1, 0, 0, 0, 0, 0]:
        d.append([img, choice])
    elif choice == [0, 0, 0, 0, 1, 0, 0, 0, 0]:
        wa.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 1, 0, 0, 0]:
        wd.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 0, 1, 0, 0]:
        sa.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 0, 0, 1, 0]:
        sd.append([img, choice])
    elif choice == [0, 0, 0, 0, 0, 0, 0, 0, 1]:
        nk.append([img, choice])
    else:
        print('no matches')    

print(len(w), len(s), len(a), len(d), len(wa), len(wd), len(sa), len(sd), len(nk))

final_data = w + s + a + d + wa + wd + sa + sd + nk
shuffle(final_data)
np.save(f'training_data_unbalanced_{i}.npy', final_data)

# balancing the data
# length = min(len(w), len(s), len(a), len(d), len(wa), len(wd), len(sa), len(sd), len(nk))

# w = w[:length]
# s = s[:length]
# a = a[:length]
# d = d[:length]
# wa = wa[:length]
# wd = wd[:length]
# sa = sa[:length]
# sd = sd[:length]
# nk = nk[:length]

# final_data = w + s + a + d + wa + wd + sa + sd + nk

shuffle(final_data)
#np.save('training_data_balanced_v2.npy', final_data)


In [ ]:
%pip install tensorflow

In [ ]:
import numpy as np
import tensorboard
WIDTH = 160
HEIGHT = 120
LR = 1e-3
EPOCHS = 1
#MODEL_NAME = 'pygta5-car-fast-{}-{}-{}-epochs-300K-data.model'.format(LR, 'alexnetv2',EPOCHS)
MODEL_NAME = 'alexnet_color_30'


##def alexnet(width, height, lr, input= 1, output=9, model_name = 'alexnet_color_30'):
#model = alexnet(WIDTH, HEIGHT, LR, input=1, output=9, model_name=MODEL_NAME)
data = np.load(r'training_data_raw.npy', allow_pickle=True)

# X = np.array([i[0] for i in data]).reshape(-1, WIDTH, HEIGHT, 1)
# Y = np.array([np.array(i[1]) for i in data])  # Convert Y to a numpy array

# print("X shape: ", X.shape)
# print("Y shape: ", Y.shape)


In [4]:
print(data.shape) # (300000, 2)
print(data[0].shape) # (2,)
print(data[1].shape) # (2,)

(3837, 2)
(2,)
(2,)


In [12]:
print(data[0][0].shape) # (270, 480, 3)
print(np.array(data[0][1]).shape) # (9,)

(270, 480, 3)
(9,)


In [24]:
X = np.array([i[0] for i in data]).reshape(-1, 270, 480, 3)
Y = np.array([np.array(i[1]) for i in data])

In [29]:
X.shape


(3837, 270, 480, 3)

In [31]:
# Check the total number of elements in the original data
total_elements = np.array([i[0] for i in data]).size

# Reshape X to (160, 120, 3)
X = np.array([i[0] for i in data]).reshape(-1, 160, 120, 3)
Y = np.array([np.array(i[1]) for i in data])

ValueError: cannot reshape array of size 1491825600 into shape (160,120,3)

In [32]:
print("Total elements in original data:", total_elements)

Total elements in original data: 1491825600
